In [1]:
import pyLDAvis
import pyLDAvis.sklearn
import time
pyLDAvis.enable_notebook()

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### dataset

In [3]:
corpus_fname = './data/hashtag_test.txt' 

In [4]:
with open(corpus_fname, encoding='utf-8') as f:
    for _ in range(5):
        print(next(f))
    

팔 명지대학교 그린나래 기획봉사 농촌활동 내

아역모델 작시 zaxy 작시슈즈 엠드림즈 브랜드 ootd 패피 신발 광고모델 김하은

보이차 아지트샵 아지트샵조히

무한체력 엄마는 너가 조용하면 좀 무섭다

감사해 에보나이트 그래도빈손아니니까



In [10]:
from soynlp.utils import DoublespaceLineCorpus
corpus_doc = DoublespaceLineCorpus(corpus_fname, iter_sent=False)

In [11]:
# twitter 방식.. 이미 학습된 내용이며 평균이상의 결과물을 보이나 
# 도메인에따라 학습된 내용이 아닌 단어인 경우 결과가 좋지 않음.. 
# 최선의 방법은 도메인에 따라 학습된 것을 바탕으로 custom_tokenize를 돌려야 함..

from konlpy.tag import Twitter
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        pos = self.tagger.pos(sent)
#         pos = ['{}/{}'.format(word,tag) for word, tag in pos if tag=='Noun']
        pos = ['{}'.format(word) for word,tag in pos if tag=='Noun' and len(word)>1]
        return pos

tw_tokenizer = MyTokenizer(Twitter())
print(tw_tokenizer('테스트는 테스트로 테스트한다'))

['테스트', '테스트', '테스트']


### Convert to document-term matrix

In [12]:

%%time
tf_vectorizer = CountVectorizer(tokenizer = tw_tokenizer,
                                max_features= 100)
dtm_tf = tf_vectorizer.fit_transform(corpus_doc)
print(dtm_tf.shape)

(603108, 3000)


In [17]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer = tw_tokenizer,
                                   max_features= 100)
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus_doc)
print(dtm_tfidf.shape)

(603108, 100)
Wall time: 9min 44s


### Fit Latent Dirichlet Allocation models

In [15]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf.fit(dtm_tf)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\mediance_ssh\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=20,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [19]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\mediance_ssh\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=20,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

### Visualizing the models with pyLDAvis

In [16]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
11     15.536342        1       1  0.278301  0.065793
16      8.100577        1       2 -0.069770 -0.020900
2       7.931546        1       3 -0.069691  0.136379
7       5.029578        1       4 -0.077674  0.062829
8       4.948543        1       5  0.251150  0.123069
0       4.948473        1       6 -0.077169  0.088330
12      4.766829        1       7 -0.056882  0.068439
19      4.560352        1       8 -0.017213 -0.239792
15      4.426351        1       9 -0.094088 -0.061578
4       4.273335        1      10  0.011116 -0.296793
1       4.158770        1      11 -0.084457 -0.003588
5       4.113815        1      12 -0.085692  0.017575
14      4.078217        1      13  0.236964  0.023761
18      3.916848        1      14 -0.073775 -0.071825
3       3.472388        1      15  0.041049 -0.030060
9       3.390982        1      16 -0.085919  0.085423
13      3.276906        1      17  0.033334 -0.099052
6       3.240183        1      18 -0.040451  0.094169
17      3.019118        1      19 -0.114487  0.151357
10      2.810846        1      20  0.095356 -0.093537, topic_info=     Category           Freq  Term          Total  loglift  logprob
term                                                               
276   Default  241000.000000    그램  241000.000000  30.0000  30.0000
1506  Default  200855.000000    스타  200855.000000  29.0000  29.0000
807   Default  145812.000000    맛집  145812.000000  28.0000  28.0000
398   Default  107798.000000    네일  107798.000000  27.0000  27.0000
2490  Default   67248.000000    카페   67248.000000  26.0000  26.0000
1819  Default   53674.000000    여행   53674.000000  25.0000  25.0000
2045  Default   40851.000000    육아   40851.000000  24.0000  24.0000
1121  Default   37346.000000    부산   37346.000000  23.0000  23.0000
427   Default   30897.000000    눈썹   30897.000000  22.0000  22.0000
1324  Default   34120.000000    선물   34120.000000  21.0000  21.0000
2131  Default   70653.000000    일상   70653.000000  20.0000  20.0000
812   Default   46729.000000    맞팔   46729.000000  19.0000  19.0000
1402  Default   34493.000000   속눈썹   34493.000000  18.0000  18.0000
343   Default   32419.000000    꽃집   32419.000000  17.0000  17.0000
497   Default   24457.000000    대구   24457.000000  16.0000  16.0000
1630  Default   24771.000000    아기   24771.000000  15.0000  15.0000
2260  Default   28623.000000    제주   28623.000000  14.0000  14.0000
988   Default   21888.000000   반영구   21888.000000  13.0000  13.0000
1396  Default   42395.000000    소통   42395.000000  12.0000  12.0000
1234  Default   21746.000000    사진   21746.000000  11.0000  11.0000
218   Default   19536.000000    광주   19536.000000  10.0000  10.0000
2921  Default   20726.000000    호빠   20726.000000   9.0000   9.0000
60    Default   22278.000000    강남   22278.000000   8.0000   8.0000
1478  Default   19742.000000    스냅   19742.000000   7.0000   7.0000
2124  Default   37734.000000    일리   37734.000000   6.0000   6.0000
901   Default   17298.000000    문신   17298.000000   5.0000   5.0000
1154  Default   16584.000000    뷰티   16584.000000   4.0000   4.0000
1636  Default   18138.000000    아들   18138.000000   3.0000   3.0000
1331  Default   30316.000000    선팔   30316.000000   2.0000   2.0000
2002  Default   24913.000000    웨딩   24913.000000   1.0000   1.0000
...       ...            ...   ...            ...      ...      ...
330   Topic20    4134.064336    김해    4135.037622   3.5714  -3.6671
1744  Topic20    3119.878740    양산    3120.852027   3.5714  -3.9486
1635  Topic20    3082.978742   아동복    3083.952029   3.5714  -3.9605
2141  Topic20    2794.709668   임산부    2795.682954   3.5713  -4.0587
1021  Topic20    2661.523155    배우    2662.496442   3.5713  -4.1075
2090  Topic20    2612.432957    이천    2613.406244   3.5713  -4.1261
243   Topic20    2510.781620    구이    2511.754907   3.5713  -4.1658
2633  Topic20    2454.338163   토요일    2455.311449

In [20]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
19     10.666762        1       1 -0.340196  0.014707
13      9.775698        1       2 -0.306915 -0.000890
11      6.844888        1       3 -0.251568  0.032409
3       6.353647        1       4  0.066269 -0.155520
6       5.569563        1       5  0.071976 -0.250473
10      5.534329        1       6 -0.276298  0.016040
5       5.235394        1       7  0.103370  0.070948
17      4.796327        1       8 -0.039022  0.121785
14      4.551800        1       9  0.063343 -0.136886
8       4.474182        1      10  0.057264 -0.213129
9       4.372825        1      11 -0.040016 -0.014914
1       4.296874        1      12  0.100130 -0.004657
2       4.136253        1      13  0.070807 -0.175737
16      3.854707        1      14  0.055432  0.127398
0       3.820696        1      15  0.041587 -0.026272
12      3.762741        1      16  0.061010  0.184084
4       3.455640        1      17  0.125231  0.022524
18      3.267943        1      18  0.149437  0.043229
7       2.657306        1      19  0.144481  0.193121
15      2.572423        1      20  0.143678  0.152232, topic_info=     Category          Freq  Term         Total  loglift  logprob
term                                                             
26    Default  27804.000000    맛집  27804.000000  30.0000  30.0000
59    Default  17853.000000    여행  17853.000000  29.0000  29.0000
15    Default  14121.000000    네일  14121.000000  28.0000  28.0000
11    Default  39715.000000    그램  39715.000000  27.0000  27.0000
35    Default  11681.000000    부산  11681.000000  26.0000  26.0000
51    Default  33867.000000    스타  33867.000000  25.0000  25.0000
3     Default  10988.000000    강남  10988.000000  24.0000  24.0000
81    Default  15994.000000    카페  15994.000000  23.0000  23.0000
29    Default   8593.000000    모델   8593.000000  22.0000  22.0000
61    Default   9679.000000    오늘   9679.000000  21.0000  21.0000
42    Default   9854.000000    선물   9854.000000  20.0000  20.0000
76    Default   8686.000000    주말   8686.000000  19.0000  19.0000
80    Default  12412.000000    추천  12412.000000  18.0000  18.0000
57    Default   8547.000000    엄마   8547.000000  17.0000  17.0000
83    Default   7113.000000    커플   7113.000000  16.0000  16.0000
53    Default   7564.000000    아들   7564.000000  15.0000  15.0000
65    Default   9489.000000    육아   9489.000000  14.0000  14.0000
74    Default   7417.000000    제주   7417.000000  13.0000  13.0000
58    Default   7160.000000    여자   7160.000000  12.0000  12.0000
38    Default   7566.000000    사진   7566.000000  11.0000  11.0000
27    Default   9989.000000    맞팔   9989.000000  10.0000  10.0000
19    Default   6398.000000    대구   6398.000000   9.0000   9.0000
62    Default   5530.000000    운동   5530.000000   8.0000   8.0000
52    Default   7809.000000    아기   7809.000000   7.0000   7.0000
67    Default   6020.000000   인스타   6020.000000   6.0000   6.0000
22    Default   6299.000000   데이트   6299.000000   5.0000   5.0000
85    Default   4218.000000   케이스   4218.000000   4.0000   4.0000
36    Default   5141.000000    뷰티   5141.000000   3.0000   3.0000
10    Default   4610.000000    광주   4610.000000   2.0000   2.0000
91    Default   5353.000000    패션   5353.000000   1.0000   1.0000
...       ...           ...   ...           ...      ...      ...
0     Topic20      0.115583    가방   2609.367388  -6.3643 -11.7819
8     Topic20      0.115583    골프   2681.224394  -6.3915 -11.7819
48    Topic20      0.115583    수원   2733.535933  -6.4108 -11.7819
45    Topic20      0.115583    셀피   2929.072212  -6.4799 -11.7819
90    Topic20      0.115584   팔로우   2979.346577  -6.4969 -11.7819
68    Topic20      0.115583    인천   3013.683556  -6.5084 -11.7819
70    Topic20      0.115583  인테리어   3103.822266  -6.5378 -11.7819
80    Topic20      0.115584    추천  12412.783220  -7.9239 -11.7819
92    Topic20      0.115584   플라워   3538.758800  -6.6690 -11.78

In [21]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
11     15.536342        1       1 -0.198139  0.469145
16      8.100577        1       2  0.054108 -0.454233
2       7.931546        1       3  0.098927  0.503293
7       5.029578        1       4 -0.320188 -0.176552
8       4.948543        1       5 -0.262011  0.290869
0       4.948473        1       6  0.037819  0.353220
12      4.766829        1       7  0.485621 -0.010413
19      4.560352        1       8 -0.461134  0.049375
15      4.426351        1       9  0.201200  0.135199
4       4.273335        1      10 -0.120464  0.060710
1       4.158770        1      11 -0.488690 -0.189340
5       4.113815        1      12  0.051771 -0.161938
14      4.078217        1      13 -0.445903  0.265231
18      3.916848        1      14  0.289092 -0.175773
3       3.472388        1      15  0.458212 -0.256739
9       3.390982        1      16  0.443227  0.191971
13      3.276906        1      17  0.332542  0.384662
6       3.240183        1      18  0.242804 -0.439268
17      3.019118        1      19 -0.252464 -0.349501
10      2.810846        1      20 -0.146332 -0.489916, topic_info=     Category           Freq  Term          Total  loglift  logprob
term                                                               
276   Default  241000.000000    그램  241000.000000  30.0000  30.0000
1506  Default  200855.000000    스타  200855.000000  29.0000  29.0000
807   Default  145812.000000    맛집  145812.000000  28.0000  28.0000
398   Default  107798.000000    네일  107798.000000  27.0000  27.0000
2490  Default   67248.000000    카페   67248.000000  26.0000  26.0000
1819  Default   53674.000000    여행   53674.000000  25.0000  25.0000
2045  Default   40851.000000    육아   40851.000000  24.0000  24.0000
1121  Default   37346.000000    부산   37346.000000  23.0000  23.0000
427   Default   30897.000000    눈썹   30897.000000  22.0000  22.0000
1324  Default   34120.000000    선물   34120.000000  21.0000  21.0000
2131  Default   70653.000000    일상   70653.000000  20.0000  20.0000
812   Default   46729.000000    맞팔   46729.000000  19.0000  19.0000
1402  Default   34493.000000   속눈썹   34493.000000  18.0000  18.0000
343   Default   32419.000000    꽃집   32419.000000  17.0000  17.0000
497   Default   24457.000000    대구   24457.000000  16.0000  16.0000
1630  Default   24771.000000    아기   24771.000000  15.0000  15.0000
2260  Default   28623.000000    제주   28623.000000  14.0000  14.0000
988   Default   21888.000000   반영구   21888.000000  13.0000  13.0000
1396  Default   42395.000000    소통   42395.000000  12.0000  12.0000
1234  Default   21746.000000    사진   21746.000000  11.0000  11.0000
218   Default   19536.000000    광주   19536.000000  10.0000  10.0000
2921  Default   20726.000000    호빠   20726.000000   9.0000   9.0000
60    Default   22278.000000    강남   22278.000000   8.0000   8.0000
1478  Default   19742.000000    스냅   19742.000000   7.0000   7.0000
2124  Default   37734.000000    일리   37734.000000   6.0000   6.0000
901   Default   17298.000000    문신   17298.000000   5.0000   5.0000
1154  Default   16584.000000    뷰티   16584.000000   4.0000   4.0000
1636  Default   18138.000000    아들   18138.000000   3.0000   3.0000
1331  Default   30316.000000    선팔   30316.000000   2.0000   2.0000
2002  Default   24913.000000    웨딩   24913.000000   1.0000   1.0000
...       ...            ...   ...            ...      ...      ...
330   Topic20    4134.064336    김해    4135.037622   3.5714  -3.6671
1744  Topic20    3119.878740    양산    3120.852027   3.5714  -3.9486
1635  Topic20    3082.978742   아동복    3083.952029   3.5714  -3.9605
2141  Topic20    2794.709668   임산부    2795.682954   3.5713  -4.0587
1021  Topic20    2661.523155    배우    2662.496442   3.5713  -4.1075
2090  Topic20    2612.432957    이천    2613.406244   3.5713  -4.1261
243   Topic20    2510.781620    구이    2511.754907   3.5713  -4.1658
2633  Topic20    2454.338163   토요일    2455.311449

In [22]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics          x          y
topic                                                  
11     15.536342        1       1 -25.378643 -37.897919
16      8.100577        1       2  27.417065 -14.055921
2       7.931546        1       3  12.910391  11.467507
7       5.029578        1       4 -24.876516  24.674868
8       4.948543        1       5  24.098528  37.674042
0       4.948473        1       6 -41.961128  45.686935
12      4.766829        1       7  36.856281  13.310337
19      4.560352        1       8 -46.820923 -20.780651
15      4.426351        1       9 -15.986491  44.943020
4       4.273335        1      10   3.400874  -9.535637
1       4.158770        1      11 -11.084122   8.172390
5       4.113815        1      12 -66.239357  33.467972
14      4.078217        1      13 -46.476826  19.968157
18      3.916848        1      14   3.293493 -34.041058
3       3.472388        1      15   0.046239  29.385965
9       3.390982        1      16 -62.589211   0.639126
13      3.276906        1      17   6.046389  57.238258
6       3.240183        1      18 -19.033102 -14.749463
17      3.019118        1      19 -34.163528   1.328488
10      2.810846        1      20 -25.190996  66.930168, topic_info=     Category           Freq  Term          Total  loglift  logprob
term                                                               
276   Default  241000.000000    그램  241000.000000  30.0000  30.0000
1506  Default  200855.000000    스타  200855.000000  29.0000  29.0000
807   Default  145812.000000    맛집  145812.000000  28.0000  28.0000
398   Default  107798.000000    네일  107798.000000  27.0000  27.0000
2490  Default   67248.000000    카페   67248.000000  26.0000  26.0000
1819  Default   53674.000000    여행   53674.000000  25.0000  25.0000
2045  Default   40851.000000    육아   40851.000000  24.0000  24.0000
1121  Default   37346.000000    부산   37346.000000  23.0000  23.0000
427   Default   30897.000000    눈썹   30897.000000  22.0000  22.0000
1324  Default   34120.000000    선물   34120.000000  21.0000  21.0000
2131  Default   70653.000000    일상   70653.000000  20.0000  20.0000
812   Default   46729.000000    맞팔   46729.000000  19.0000  19.0000
1402  Default   34493.000000   속눈썹   34493.000000  18.0000  18.0000
343   Default   32419.000000    꽃집   32419.000000  17.0000  17.0000
497   Default   24457.000000    대구   24457.000000  16.0000  16.0000
1630  Default   24771.000000    아기   24771.000000  15.0000  15.0000
2260  Default   28623.000000    제주   28623.000000  14.0000  14.0000
988   Default   21888.000000   반영구   21888.000000  13.0000  13.0000
1396  Default   42395.000000    소통   42395.000000  12.0000  12.0000
1234  Default   21746.000000    사진   21746.000000  11.0000  11.0000
218   Default   19536.000000    광주   19536.000000  10.0000  10.0000
2921  Default   20726.000000    호빠   20726.000000   9.0000   9.0000
60    Default   22278.000000    강남   22278.000000   8.0000   8.0000
1478  Default   19742.000000    스냅   19742.000000   7.0000   7.0000
2124  Default   37734.000000    일리   37734.000000   6.0000   6.0000
901   Default   17298.000000    문신   17298.000000   5.0000   5.0000
1154  Default   16584.000000    뷰티   16584.000000   4.0000   4.0000
1636  Default   18138.000000    아들   18138.000000   3.0000   3.0000
1331  Default   30316.000000    선팔   30316.000000   2.0000   2.0000
2002  Default   24913.000000    웨딩   24913.000000   1.0000   1.0000
...       ...            ...   ...            ...      ...      ...
330   Topic20    4134.064336    김해    4135.037622   3.5714  -3.6671
1744  Topic20    3119.878740    양산    3120.852027   3.5714  -3.9486
1635  Topic20    3082.978742   아동복    3083.952029   3.5714  -3.9605
2141  Topic20    2794.709668   임산부    2795.682954   3.5713  -4.0587
1021  Topic20    2661.523155    배우    2662.496442   3.5713  -4.1075
2090  Topic20    2612.432957    이천    2613.406244   3.5713  -4.1261
243   Topic20    2510.781620    구이    2511.754907   3.5713  -4.1658
2633 